In [1]:
import json
import os
import _jsonnet
import os

In [2]:
from seq2struct.commands.infer import Inferer
from seq2struct.datasets.spider import SpiderItem
from seq2struct.utils import registry

In [3]:
import torch

In [4]:
exp_config = json.loads(
    _jsonnet.evaluate_file(
        "experiments/spider-configs/spider-mT5-large-en-pt-es-fr-train_en-pt-es-fr-eval.jsonnet"))


In [5]:
model_config_path = exp_config["model_config"]
model_config_args = exp_config.get("model_config_args")

In [6]:
infer_config = json.loads(
    _jsonnet.evaluate_file(
        model_config_path, 
        tla_codes={'args': json.dumps(model_config_args)}))

In [7]:
infer_config["model"]["encoder_preproc"]["db_path"] = "data/sqlite_files/"

In [8]:
inferer = Inferer(infer_config)

WARNING <class 'seq2struct.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}


SpiderEncoderT5Preproc Model: google/mt5-large
No GAP - Generation-Augmented Pre-Training
T5 load Model: google/mt5-large


In [9]:
model_dir = exp_config["logdir"] + "/bs=4,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1"
#checkpoint_step = exp_config["eval_steps"][0]
checkpoint_step = 51100

In [10]:
model_dir

'/mnt/Databases/nl2sql/gap-text2sql/mrat-sql-gap/logdir/mT5-large-en-pt-es-fr-train/bs=4,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1'

In [11]:
model = inferer.load_model(model_dir, checkpoint_step)

WARNING <class 'seq2struct.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'clause_order': None, 'end_with_from': True, 'factorize_sketch': 2, 'include_literals': False, 'infer_from_conditions': True, 'name': 'spider', 'output_from': True, 'use_table_pointer': True}, 'save_path': 'data/spider-en-pt-es-fr/mT5-large-nl2code-1115,output_from=true,fs=2,emb=t5,cvlink', 'use_seq_elem_rules': True}, 'encoder_preproc': {'compute_cv_link': True, 'compute_sc_link': True, 'db_path': 'data/sqlite_files/', 'fix_issue_16_primary_keys': True, 'include_table_name_in_column': False, 'pretrained_checkpoint': 'models/mt5-large/pretrained_checkpoint/pytorch_model.bin', 'save_path': 'data/spider-en-pt-es-fr/mT5-large-nl2code-1115,output_from=true,fs=2,emb=t5,cvlink', 't5_version': 'google/mt5-large'}}


SpiderEncoderT5 Model: google/mt5-large
No GAP - Generation-Augmented Pre-Training


Some weights of the model checkpoint at google/mt5-large were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Parameter containing:
tensor([[-1.0312e+00, -4.2500e+00,  7.0000e+00,  ...,  6.0938e+00,
         -8.0625e+00, -9.5000e+00],
        [-7.7500e+00, -1.2125e+01, -2.3438e+00,  ..., -7.8438e+00,
          9.1875e+00,  4.4375e+00],
        [ 9.8047e-01,  1.0781e+00, -3.8672e-01,  ..., -1.0156e+00,
         -4.7852e-01,  8.0078e-01],
        ...,
        [-1.4062e+00, -5.3516e-01,  1.7031e+00,  ..., -1.2891e+00,
         -1.6504e-01, -1.0078e+00],
        [-2.6953e-01,  6.2500e-01, -1.3828e+00,  ...,  1.8750e+00,
          5.7373e-02,  1.4160e-01],
        [-6.7188e-01,  5.8203e-01, -3.1250e-01,  ...,  5.3711e-03,
         -2.1289e-01,  4.8828e-01]], requires_grad=True)
Parameter containing:
tensor([[ -1.0312,  -4.2500,   7.0000,  ...,   6.0938,  -8.0625,  -9.5000],
        [ -7.7500, -12.1250,  -2.3438,  ...,  -7.8438,   9.1875,   4.4375],
        [  0.9805,   1.0781,  -0.3867,  ...,  -1.0156,  -0.4785,   0.8008],
        ...,
        [  0.8594,   0.1523,   0.1260,  ...,   1.2188,  -0.5195

In [12]:
from seq2struct.datasets.spider_lib.preprocess.get_tables import dump_db_json_schema
from seq2struct.datasets.spider import load_tables_from_schema_dict

In [13]:
#db_id = "singer"
#db_id = "concert_singer"
db_id ="hospital_1"

In [14]:
my_schema = dump_db_json_schema("data/sqlite_files/{db_id}/{db_id}.sqlite".format(db_id=db_id), db_id)
#my_schema = dump_db_json_schema("data/spider-en/database/{db_id}/{db_id}.sqlite".format(db_id=db_id), db_id)

In [15]:
from seq2struct.utils.api_utils import refine_schema_names

In [16]:
my_schema

{'db_id': 'hospital_1',
 'table_names_original': ['Physician',
  'Department',
  'Affiliated_With',
  'Procedures',
  'Trained_In',
  'Patient',
  'Nurse',
  'Appointment',
  'Medication',
  'Prescribes',
  'Block',
  'Room',
  'On_Call',
  'Stay',
  'Undergoes'],
 'table_names': ['physician',
  'department',
  'affiliated with',
  'procedures',
  'trained in',
  'patient',
  'nurse',
  'appointment',
  'medication',
  'prescribes',
  'block',
  'room',
  'on call',
  'stay',
  'undergoes'],
 'column_names_original': [(-1, '*'),
  (0, 'EmployeeID'),
  (0, 'Name'),
  (0, 'Position'),
  (0, 'SSN'),
  (1, 'DepartmentID'),
  (1, 'Name'),
  (1, 'Head'),
  (2, 'Physician'),
  (2, 'Department'),
  (2, 'PrimaryAffiliation'),
  (3, 'Code'),
  (3, 'Name'),
  (3, 'Cost'),
  (4, 'Physician'),
  (4, 'Treatment'),
  (4, 'CertificationDate'),
  (4, 'CertificationExpires'),
  (5, 'SSN'),
  (5, 'Name'),
  (5, 'Address'),
  (5, 'Phone'),
  (5, 'InsuranceID'),
  (5, 'PCP'),
  (6, 'EmployeeID'),
  (6, 'Na

In [17]:
# If you want to change your schema name, then run this; Otherwise you can skip this.
#refine_schema_names(my_schema)

In [18]:
schema, eval_foreign_key_maps = load_tables_from_schema_dict(my_schema)

In [19]:
schema.keys()

dict_keys(['hospital_1'])

In [20]:
dataset = registry.construct('dataset_infer', {
   "name": "spider", "schemas": schema, "eval_foreign_key_maps": eval_foreign_key_maps, 
    "db_path": "data/sqlite_files/"
})

In [21]:
for _, schema in dataset.schemas.items():
    model.preproc.enc_preproc._preprocess_schema(schema)

In [22]:
spider_schema = dataset.schemas[db_id]

In [23]:
def infer(question):
    data_item = SpiderItem(
            text=None,  # intentionally None -- should be ignored when the tokenizer is set correctly
            code=None,
            schema=spider_schema,
            orig_schema=spider_schema.orig,
            orig={"question": question}
        )
    model.preproc.clear_items()
    enc_input = model.preproc.enc_preproc.preprocess_item(data_item, None)
    preproc_data = enc_input, None
    with torch.no_grad():
        output = inferer._infer_one(model, data_item, preproc_data, beam_size=1, use_heuristic=True)
    #print(f"orig_question: {output[0]['orig_question']}")
    #print(f"model_output: {output[0]['model_output']}")
    #print(f"score: {output[0]['score']}") 
    return output[0]["inferred_code"]

In [ ]:
#aguarda pergunta e responde a resposta (inferência)

import csv
import time
import os


path = "/mnt/Files/integration/nl2sql/"
print(f"Listening path: {path}")

while 1:#loop infinito tem que parar manualmente
    time.sleep(0.5)#em segundos, delay para não consumir todo o processador da máquina.
    
    ready = os.path.exists(path + "in/ready.txt")
    question = os.path.exists(path + "in/question.csv")
    #print(f"Ready = {ready}\nQuestion = {question}")
        
    if ready == True and question == True:
        print("\nNew request arrive:")
                
        csv_out = open(path + "out/queries.csv", "w", encoding='utf8')
        ready_out = open(path + "out/ready.txt", "w", encoding='utf8')
        ready_out.write("Done")#qualquer conteúdo       
       
        
        with open(path + "in/question.csv", newline = '', encoding='utf8') as csv_in:
            csv_in_reader = csv.reader(csv_in, delimiter=';')
            for csv_i in csv_in_reader:
                print(csv_i[0])
                code = infer(csv_i[0])               
                print(code)
                csv_out.write(f"{csv_i[0]};{code}\n")
                break #só o primeiro               
        csv_out.close()   
        ready_out.close()
        os.remove(path + "in/ready.txt") #Deletado indica que terminou o processamento
        os.remove(path + "in/question.csv")#Deletado indica que terminou o processamento

Listening path: /mnt/Files/integration/nl2sql/

New request arrive:
Encontre os nomes das enfermeiras que estão de plantão.


/home/marchanjo/miniconda3/envs/gap-text2sql/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


SELECT DISTINCT Nurse.Name FROM Nurse JOIN On_Call

New request arrive:
Encontre os nomes das enfermeiras que estão de plantão.
SELECT DISTINCT Nurse.Name FROM Nurse JOIN On_Call

New request arrive:
Qual departamento possui o maior número de funcionários?
SELECT Department.Name FROM Department GROUP BY Department.DepartmentID ORDER BY Count(Department.DepartmentID) Desc LIMIT 1

New request arrive:
Qual departamento possui o maior número de funcionários?
SELECT Department.Name FROM Department GROUP BY Department.DepartmentID ORDER BY Count(Department.DepartmentID) Desc LIMIT 1

New request arrive:
Encontre os três procedimentos mais caros
SELECT Procedures.Name FROM Procedures ORDER BY Procedures.Cost Asc LIMIT 1

New request arrive:
Encontre os 3 procedimentos mais caros
SELECT Procedures.Name FROM Procedures ORDER BY Procedures.Cost Asc LIMIT 1
